<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/Model_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.6.2)


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek

Using TensorFlow backend.


In [0]:
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

In [0]:
df2 = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df2 = df2[df2['attribute_name'] != 'style'].drop(columns = ['attribute_name'])
df2['attribute_value'] = 0
df2 = df2.drop_duplicates()

In [5]:
df = pd.concat([df, df2])
df['attribute_value'].value_counts()

0                 48077
casual             2633
modern             1838
classic            1488
businesscasual     1484
edgy                804
androgynous         691
romantic            558
boho                468
glam                423
athleisure          280
retro               220
Name: attribute_value, dtype: int64

In [0]:
df = pd.get_dummies(df, columns=['attribute_value'])
df = df.groupby(['product_id', 'brand', 'product_full_name', 'description', 'brand_category']).sum().reset_index()
label_list = []
for key in df.columns:
    if 'attribute_value' in key:
        df[key] = df[key].apply(lambda x: min(x, 1))
        label_list.append(key)
y = df[label_list].values
num_classes = y.shape[1]

In [0]:
X = df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']
X = X.values

In [0]:
vectorizer = TfidfVectorizer(max_features=500, min_df = 5, ngram_range = (1, 2))

In [0]:
X = vectorizer.fit_transform(X)

In [0]:
scaler = StandardScaler( with_mean = False)
X = scaler.fit_transform(X,)

In [11]:
resampler = SMOTETomek(sampling_strategy = 0.1)
X, y = resampler.fit_resample(X, y)

ValueError: ignored

In [0]:
X.reshape(-1,500)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

def define_model():
    inputs = Input(shape = (500, ))
    embedding1_1 = Embedding(2000, 100)(inputs)
    conv1_1 = Conv1D(filters=32, kernel_size=10, activation='tanh')(embedding1_1)

    pool1_1 = MaxPooling1D(pool_size=2)(conv1_1)
    flatten = Flatten()(pool1_1)

    dense1 = Dense(64, activation = 'tanh')(flatten)
    outputs = Dense(num_classes, activation='sigmoid')(dense1)

    

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.03), metrics=['accuracy'])

    return model

In [0]:
def define_model():
    inputs = Input(shape=(500,))
    embedding = Embedding(input_dim=500, output_dim=100)(inputs)
    conv = Conv1D(filters = 32, kernel_size=10, activation='relu')(embedding)
    pool = MaxPooling1D(pool_size=2)(conv)
    flatten = Flatten()(pool)
    dense = Dense(64, activation='relu')(flatten)
    outputs = Dense(num_classes, activation='sigmoid')(dense)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

def define_model():
    model = Sequential()
    model.add(Embedding(input_dim=200, output_dim=100, input_length=500))
    model.add(Conv1D(filters=32, kernel_size=10, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(64,activation='relu'))
    model.add(Dense(11, activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [0]:
model = define_model()
model.summary()

In [0]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=512, epochs=10)

In [0]:
model.evaluate(X_test, y_test)

In [0]:
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()